# Bllossom-8B 양자화 모델을 이용한 한국어 RAG

In [1]:
!CMAKE_ARGS="-DGGML_BLAS=ON -DGGML_BLAS_VENDOR=OpenBLAS" pip install --upgrade --verbose --force-reinstall --no-cache-dir llama-cpp-python #양자화 구동을 위한 Llama C++ 설치
# conda install -c conda-forge gcc gxx
# !huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./' #Bllossom모델 다운로드

In [1]:
from llama_cpp import Llama
import transformers
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

/home/choiyt3465/anaconda3/envs/dnlab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_emb

In [3]:
from transformers import AutoModel, AutoTokenizer
import json
from datetime import datetime
import chromadb

COLLECTION_NAME = "cnu_post_data"
COLLECTION_NAME2 = "cnu_post_content_data"

client = chromadb.PersistentClient()
collection = client.get_or_create_collection(name=COLLECTION_NAME)
collection2 = client.get_or_create_collection(name=COLLECTION_NAME2)

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')

In [60]:
single_query = '다른 학과로 전과하고 싶은데 조건이 있을까?'

documents = list()
documents.append(single_query)

inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)
doc_embeddings = []

for embedded_doc in embeddings:
    doc_embedding = embedded_doc[0].detach().cpu().numpy().tolist()
    doc_embeddings.append(doc_embedding)

LIMIT = 15

query_result = collection2.query(
            query_embeddings=doc_embeddings,
            where={"학과명": {"$in": ["컴퓨터융합학부","인공지능학과"]}},
            n_results=LIMIT)

for idx,result_documents in enumerate(query_result["documents"][0]):
        print(idx,result_documents)

0 [주의사항] 복수(부)전공 학과로 전과한 경우복수(부)전공 취소신청을,교직과정 이수자가 전과한 경우원 소속 학과의교직과정 이수자격 취소신청을 하여야 함  전과한 자 중 복수(부)전공 및 교직과정 포기자는 제2학기 융복합창의전공 신청 시기에 반드시 취소 신청하여야 함  전과는 재학 중 1회에 한함   전과한 자는 전과 전후 학과의 성적우수, 격려 장학생으로 선발 불가 * 전과하고자 하는 과의 선 이수과목 등 요구사항을 꼭 확인하세요. * 자세한 내용은 첨부문서 참고 내용 일정 비고 전과 시행계획 및 모집요강 안내 2023. 12. 8.( 금 ) 학사지원과 전과 지원서류 접수 2024. 1. 12.( 금 )  1. 16.( 화 ) 현 소속 학과 전과 ( 전출 ) 지원자 명단 제출 2024. 1. 17.( 수 ) 단과대학 ( 학과 ) 전과 지원서류 이송 2024. 1. 17.( 수 ) 단과대학 ( 학과 ) 전과 ( 전입 ) 면접고사 , 필기고사 및 실기고사 2024. 1. 19.( 금 ) 10:00 단과대학
1 - 수업 (수강신청, 졸업사정 등 ) 문의:  042-821-7456 - 학생 (장학, 졸업프로젝트, 휴복학 등) 문의:  042-821-6651 - 대학원 진학 문의: 042-821-5441 - 수강신청, 졸업사정 상담은 학과로 연락 또는 방문 하여 상담 하시기 바랍니다. (이메일로 상담 하지 않음)
2 카카오테크 캠퍼스 2기 신청자는 2024학년도 하기 계절학기 전공 수강희망서(실무프로젝트랩2) 로 제출하여 주시기 바랍니다. 2024 학년도 하기 계절학기 개설 교과목 수요조사 안내 구 분  내 용 수요조사 및 교과목 설강 기준 졸업학점 미달 , 재이수 및 조기졸업 희망 학생 등 사전 수요 조사 를 통하여  수강 희망 인원이 전공과목은 20 명 이상 ,  교양 교직 일선 과목은 30 명 이상  되어야 설강 대상자  성적우수자로 조기졸업 희망자  학점 미취득과목 취득희망자  성적 불량과목 (F 학점 포함 ) 재이수 희망자  기타 학점취득 희망자  2024

In [61]:

torch.cuda.empty_cache()

PROMPT = \
    f'''You are a helpful AI assistant. You must respond to user queries logicaly, kindly and accurately in korean.
    Refer to the searched documents to answer the user query. retrieved documents: {query_result["documents"][0]}
    If the information from the retrieved documents is insufficient, respond with "Insufficient information available." and
    When a document have a time information, you must mention it correctly, Today is {'2024-02-12'}
    '''

instruction = f'{single_query}'

messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{instruction}"}
]

prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
)

generation_kwargs = {
        "max_tokens":2048,
        "stop":["<|eot_id|>"],
        "top_p":0.9,
        "temperature":0.75,
        "echo":True, # Echo the prompt in the output
}

terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

outputs = pipeline(
        prompt,
        max_new_tokens=1024,
        min_new_tokens=128,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

# Answer
answer = outputs[0]["generated_text"][len(prompt):]
print(answer)

다른 학과로 전과하고 싶으신 경우, 몇 가지 조건을 확인해야 합니다. 아래는 전과 전형에 대한 주요 사항입니다.

1. **전과 전형 시행 계획 및 모집 요강 안내**:
   - 전과 전형은 학기마다 시행 계획이 달라질 수 있으므로, 매 학기마다 학사지원과에서 확인해야 합니다.
   - 2024학년도 1학기 전과 전형 일정은 2023년 12월 8일에 안내되었으며, 지원서는 2024년 1월 12일부터 1월 16일까지 접수되었습니다.

2. **전과 전형 요건**:
   - 전과는 재학 중 1회에 한하여 가능합니다.
   - 전과 전형에 지원하려면, 원 소속 학과의 교직과 이수자격 취소신청이 필요합니다.

3. **전과 전형 절차**:
   - 전과 지원자는 전과 전형 시행 계획 및 모집 요강을 확인한 후, 전과 전형 신청서류를 제출해야 합니다.
   - 전과 전형은 필기고사, 면접고사, 실기고사 등의 절차를 거칠 수 있습니다.

4. **전과 전형 지원서류**:
   - 전과 전형 지원서
   - 전과 전형 지원서 첨부서 (원 소속 학과의 교직과 이수자격 취소신청서, 성적증명서, 교수 회의록 등)
   - 전과 전형 지원자는 전과 전형 지원서류를 정확히 제출해야 하며, 미제출 또는 잘못된 서류로 인한 전형 탈락을 받을 수 있습니다.

5. **전과 전형 일정**:
   - 전과 전형 일정은 학기별로 다를 수 있으므로, 매 학기마다 학사지원과에서 확인해야 합니다.
   - 2024학년도 1학기 전과 전형 일정은 다음과 같습니다.
     - 전과 지원서류 접수: 2024년 1월 12일 ~ 1월 16일
     - 전과 전형 면접고사 및 실기고사: 2024년 1월 17일
     - 전과 전형 결과 발표: 2024년 1월 26일

6. **전과 전형 관련 문의**:
   - 전과 전형에 대한 자세한 문의는 학사지원과로 연락하시기 바랍니다.
   - 학사지원과 연락처: 042-821-7456

전과 전형에 대한 자세한 내용은 학사지원과에서 확인하셔서 정확한 정보를 얻